## ch02 自动求梯度


In [1]:
import torch

In [3]:
# 创建一个2*2的tensor
x=torch.ones(2,2,requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


x.grad_fn, 如果该tensor是由某些运算得到的，就会返回一个与运算相关的对象，否则该值就是None

In [4]:
y=x+2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


直接创建的tensor，称为叶子结点，叶子结点对应的grad_fn为None

In [7]:
print(x.is_leaf)
print(y.is_leaf)

True
False


### 梯度

令 $y=x+2$,

令$z=3y^2$ ，

求取一个标量$o=z.mean()$，即$o=\frac{1}{4}z$

> 实际上是赋予了每个数据，$\frac{1}{n}$的权重。其中n为数据总数。

现在开始求取x的梯度(这里的梯度，和导数是一同个东西。（感觉）)

当x的值为1，得到求导值$\frac{dy}{dx}|_{x=1}=6$

于是：

$$
\frac{do}{dx}|_{x=1,1,1,1}=\\
\frac{do}{dz}*\frac{dz}{dy}*\frac{dy}{dx}|_{x=1,1,1,1}=\\
\frac{1}{4} * 6y * 1|_{x=1,1,1,1}=\\
\frac{3}{2} * (x+2)|_{x=1,1,1,1}=\{4.5,4.5,4.5,4.5\}
$$

Note:遵从链式求导法则。

代码：

In [8]:
z=y*y*3

# 求均值
out=z.mean()
print(out)

tensor(27., grad_fn=<MeanBackward0>)


In [9]:
# 求梯度

# out是一个标量，所以不需要指定求导变量
out.backward()

# x.grad
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


### 求梯度，示例2

In [2]:
x=torch.tensor([1.0,2.0,3.0,4.0],requires_grad=True)

y=2*x*x
z=y.view(2,2)
print(z)

tensor([[ 2.,  8.],
        [18., 32.]], grad_fn=<ViewBackward>)


In [3]:
out=z.mean()
print(out)

tensor(15., grad_fn=<MeanBackward0>)


In [4]:
out.backward()
print(x.grad)

tensor([1., 2., 3., 4.])


### 求梯度，示例3

In [5]:
x=torch.tensor([1.0,2.0,3.0,4.0],requires_grad=True)

y=2*x
z=y.view(2,2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [6]:
# z不是一个标量，
# 所以需要传入一个和z同型的权重向量，
# 进行加权，来得到一个标量
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)

# backward()
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


可见，x与x_grad是同形的张量。

### 中断梯度

使用语句：

```python
y1=x**2
with torch.no_grad():
    y2=x**3

y3=y1+y2

```
> 上述代码中，求y3对于x的梯度，实际上是求y1对于x的梯度，

y2会被直接忽略。


示例：

In [11]:
x=torch.tensor(1.0,requires_grad=True)
y1=x**2
with torch.no_grad():
    y2=x**3
y3=y1+y2

print(x.requires_grad)
print(y1,y1.requires_grad) # true
print(y2,y2.requires_grad) # false 
print(y3,y3.requires_grad) # true

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [12]:
y3.backward()
print(x.grad)

tensor(2.)


**梯度清零**

grad在方向传播的过程中，是累加的。
所以在反向传播之前，要将之前的梯度清零。

> 带_的参数，等价于inplace=true，即直接替换原数据。

代码：

In [13]:
x.grad.data.zero_()

tensor(0.)

**tensor.data()**

如果修改tensor的值，但不想被autograd记录，可以采用tensor.data()对数据进行操作。

> tensor.data*=100，则原数据x直接被修改。

In [21]:
x=torch.ones(3,3,requires_grad=True)

print("x:\n",x)
print("x.data\n",x.data)
print("x.data.requires_grad\n",x.data.requires_grad)

x:
 tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
x.data
 tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
x.data.requires_grad
 False


In [22]:
y=2*x

x.data*=100 #不会记录在计算图中
print(x.data)

tensor([[100., 100., 100.],
        [100., 100., 100.],
        [100., 100., 100.]])


In [38]:
weights=torch.rand(3,3)

# norm
weights/=weights.sum()


y.backward(weights)
print(x)
print(x.grad)

tensor([[100., 100., 100.],
        [100., 100., 100.],
        [100., 100., 100.]], requires_grad=True)
tensor([[0.5563, 0.1398, 0.1841],
        [0.0857, 0.0010, 0.6584],
        [0.0203, 0.1892, 0.1654]])
